In [33]:
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.api import local_qvm
import random
from math import pi

# Prepare mixed states that are 0 with probability p
def randomize(prog,regs, p):
    for i in regs:
        if random.random() > p:
            prog+=X(i)
            
def cry(prog, control, target, theta):
    prog += RY(theta/2, target)
    prog += CNOT(control, target)
    prog += RY(-1*theta/2, target)
    prog += CNOT(control, target)
    
# Rigetti gives results as a dictionary with values of arrays for each qubit
# This reformats it so you see the result of each run instead
def reformat(results, qubits = 0):
    if qubits ==0:
        qubits = len(results)
    trials = len(results[0])
    arr = []
    for i in range(trials):
        string = ""
        for j in range(qubits):
            string += str(results[j][i])
        arr.append(string)
    return arr

def normalize(data):
    total = sum(data.values())
    for i in data.keys():
        data[i] = data[i]/total
    return data

trials = 200
pval = 0.7
log_qubits = 2

cleanData = {}
for i in range(2**log_qubits):
    cleanData[str(bin(i)[2:].zfill(log_qubits))] = 0

for i in range(trials):
    p = Program()
    
    # Used to create "mixed states"
    randomize(p,[1,3],pval)
    
    
    # Logical controlled-NOT from spin s to momentum m
    p += CNOT(3,2)
    # Logical controlled-Y rotation from J,m to s
    cry(p,1,3,pi/2)
    cry(p,2,3,pi/2)
    # Logical controlled-NOT from spin s to J
    p += CNOT(3,0)

    
    qc = get_qc('9q-square-qvm')
    result = qc.run_and_measure(p,trials=1)
    
    # The number 4 is dependent on fact that we are doing two qubits
    cleanResult = reformat(result,4)
    for string in cleanResult:
        if string == '0000':
            cleanData['11']+=1
        elif string == '0010':
            cleanData['10']+=1
        elif string == '0100':
            cleanData['10']+=1
        elif string == '0110':
            cleanData['01']+=1
        elif string == '1101':
            cleanData['00']+=1
        elif string == '1011':
            cleanData['00']+=1
    
print(normalize(cleanData))


{'00': 0.22, '01': 0.095, '10': 0.235, '11': 0.45}
